# 2008 Olympic Medalists

## Data Tidying

In [70]:
# Let's read in the data 

import pandas as pd

olympics = pd.read_csv("/Users/jordynsmall/Documents/Small-Python-Portfolio/TidyData-Project/olympics_08_medalists.csv")

olympics.head() # showing the first 5 rows of data 

# Initial observations: Many of the column names have two variables (i.e - Gender and sport, hence why there are so many nulls)

,medalist_name,male_archery,female_archery,male_athletics,female_athletics,male_badminton,female_badminton,male_baseball,male_basketball,female_basketball,...,female_beach volleyball,male_volleyball,female_volleyball,male_water polo,female_water polo,male_weightlifting,female_weightlifting,male_freestyle wrestling,female_freestyle wrestling,male_greco-roman wrestling
0,Aaron Armstrong,NaN,NaN,gold,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Peirsol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abdullo Tangriev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abeer Abdelrahman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,bronze,NaN,NaN,NaN
4,Abhinav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
olympics.describe()

# There are 1875 medalists in our dataset over 70 different sports

,medalist_name,male_archery,female_archery,male_athletics,female_athletics,male_badminton,female_badminton,male_baseball,male_basketball,female_basketball,...,female_beach volleyball,male_volleyball,female_volleyball,male_water polo,female_water polo,male_weightlifting,female_weightlifting,male_freestyle wrestling,female_freestyle wrestling,male_greco-roman wrestling
count,1875,11,9,85,81,12,10,72,35,36,...,6,36,36,39,39,24,21,28,16,27
unique,1875,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
top,Aaron Armstrong,bronze,bronze,silver,silver,silver,silver,silver,bronze,bronze,...,gold,bronze,gold,silver,gold,gold,bronze,bronze,bronze,bronze
freq,1,4,4,31,28,4,4,24,12,12,...,2,12,12,13,13,8,7,14,8,13


In [72]:
# We want to bring all of the sports into one column and have the medal classification in another column

olympics_melted = pd.melt(olympics,
        id_vars = "medalist_name", # keeping names the same
        value_vars = olympics.columns.difference(["medalist_name"]), # pivoting everything except first column
        var_name = "Sport", # name of new columns
        value_name = "Medal") # where the values go

olympics_melted.head()

,medalist_name,Sport,Medal
0,Aaron Armstrong,female_archery,NaN
1,Aaron Peirsol,female_archery,NaN
2,Abdullo Tangriev,female_archery,NaN
3,Abeer Abdelrahman,female_archery,NaN
4,Abhinav,female_archery,NaN


In [73]:
olympics_melted[["Gender","Sport"]] = olympics_melted['Sport'].str.split("_", expand=True)

olympics_melted.head()

,medalist_name,Sport,Medal,Gender
0,Aaron Armstrong,archery,NaN,female
1,Aaron Peirsol,archery,NaN,female
2,Abdullo Tangriev,archery,NaN,female
3,Abeer Abdelrahman,archery,NaN,female
4,Abhinav,archery,NaN,female


In [74]:
# The dataset looks tidy, but there are lot of null values... probably because a person is listed under each sport 

# Let's check for duplicates 

duplicates = olympics_melted[olympics_melted.duplicated(subset="medalist_name")]
duplicates

# Earlier, we knew that there were 1875 people in our dataset, now there are so many more

,medalist_name,Sport,Medal,Gender
1875,Aaron Armstrong,artistic gymnastics,NaN,female
1876,Aaron Peirsol,artistic gymnastics,NaN,female
1877,Abdullo Tangriev,artistic gymnastics,NaN,female
1878,Abeer Abdelrahman,artistic gymnastics,NaN,female
1879,Abhinav,artistic gymnastics,NaN,female
...,...,...,...,...
131245,Éver Banega,weightlifting,NaN,male
131246,Ólafur Stefánsson,weightlifting,NaN,male
131247,Óscar Brayson,weightlifting,NaN,male
131248,Łukasz Pawłowski,weightlifting,NaN,male


In [75]:
# Let's remove these null values 

olympics_melted = olympics_melted.dropna()

olympics_melted

# Awesome, we now have 1875 medalists again!

,medalist_name,Sport,Medal,Gender
249,Bérengère Schuh,archery,bronze,female
285,Chen Ling,archery,silver,female
597,Guo Dan,archery,silver,female
781,Joo Hyun-Jung,archery,gold,female
1316,Park Sung-hyun,archery,silver,female
...,...,...,...,...
131074,Vencelas Dabaya,weightlifting,silver,male
131087,Viktors Ščerbatihs,weightlifting,bronze,male
131180,Yoandry Hernández,weightlifting,bronze,male
131188,Yordanis Borrero,weightlifting,bronze,male
